# Install required library

In [ ]:
!pip show transformer

In [ ]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         0.34.2
aiohappyeyeballs                   2.4.2
aiohttp                            3.10.8
aiosignal                          1.3.1
alabaster                          0.7.16
albucore                           0.0.16
albumentations                     1.4.15
altair                             4.2.2
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.5.1
arviz                              0.19.0
astropy                            6.1.4
astropy-iers-data                  0.2024.9.30.0.32.59
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic                           4.1.0
attrs                              24.2.0
audioread         

In [ ]:
!pip install trl peft accelerate datasets bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset,Dataset,DatasetBuilder
from trl import SFTTrainer
from accelerate import Accelerator
import os
from peft import LoraConfig,AutoPeftModelForCausalLM, prepare_model_for_kbit_training,get_peft_model
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,TrainingArguments

In [ ]:
sft_config={

            "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "sft-tiny-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5

}

In [ ]:
class TrainSFT:

  def __init__(self,data,config):
    self.data=data
    self.config=config

  def prepare_lora_model(self):
    self.lora_config = LoraConfig(r=self.config["r"],
                                         lora_alpha=self.config["lora_alpha"],
                                         lora_dropout=self.config["lora_dropout"],
                                         bias=self.config["bias"],
                                         task_type=self.config["task_type"],
                                         target_modules=self.config["target_modules"])

    self.model = get_peft_model(self.model,self.lora_config)

  def load_model_tokenizer(self):
    self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"])

    self.model.config.use_cache = False
    self.model.config.pretraining_tp=1
    self.model = prepare_model_for_kbit_training(self.model)


    if self.config["use_lora"]:
      self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
    self.tokenizer.pad_token = self.tokenizer.eos_token


  def set_training_args(self):
     return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

  # Implemented trainer
  def create_trainer(self):
        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )


  def train_and_save_model(self):
    # Creating trainer
    self.create_trainer()

    #Train
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])






In [ ]:

  # Creating data
  def create_data():
    data = load_dataset("tatsu-lab/alpaca",split="train")
    data_df = data.to_pandas()
    data_df = data_df[:700]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

  data = create_data()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_sft = TrainSFT(data,sft_config)
train_sft.train_and_save_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
None


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow

Step,Training Loss
100,2.179600
200,1.752100


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


events.out.tfevents.1727924883.b6eac170bc0d.7633.0:   0%|          | 0.00/6.58k [00:00<?, ?B/s]

In [ ]:
!pip install transformers --upgrade

# **Preference Aligment-DPO**

In [ ]:
dpo_config = {
            "model_ckpt": "snshrivas10/sft-tiny-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "tiny-chatbot-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto_pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

# You can use below config for local model load instead of hugging face hub
#   "model_ckpt": "/content/sft-tiny-chatbot"

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import DPOTrainer
import pandas as pd
from accelerate import Accelerator
from peft import get_peft_model, LoraConfig

In [ ]:
class TrainDPO:

  def __init__(self,data,config):
    self.data=data
    self.config=config

  def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
        self.model_ref = get_peft_model(self.model_ref, self.lora_config)

  def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

  def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )


  def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
        else:
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        peft_config=self.lora_config,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config["output_dir"])
    self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data
data = create_data()

In [ ]:
train_dpo = TrainDPO(data, dpo_config)
train_dpo.train_and_save_model()

# Inferencing

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("snshrivas10/tiny-chatbot-dpo")

In [ ]:
model

In [ ]:
import tourch

In [ ]:
from transformers import pipeline

In [ ]:
pipe=pipeline("text-generation",model=model,tokenizer="snshrivas10/tiny-chatbot-dpo",torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many aeroplane can a human eat in one sitting?"},
]

In [ ]:
prompt = pipe.tokenizer.apply_chat_template(messages,tokenoxer=False,add_generation_prompt=True)

In [ ]:
prompt

In [ ]:
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

In [ ]:
outputs[0]["generated_text"]